In [5]:
import random
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
import math,cmath
from scipy.sparse import diags
from scipy.integrate import quad
from mpmath import *
from scipy import integrate
from scipy import optimize

In [10]:
Nmst = 60; #Number of lattice points
Pbst = 30;#Probe attachment site
lbd = 0.0 ;#Lambda strength ofAAH
irrb = (1 + np.sqrt(5))/2
t = 1.0; # hopping potential for sites
to = 3.0; # hopping potential for bath
sitegammaindx = [0, Nmst-1, Pbst-1]
sitegammastrn = [1.0, 1.0, 0.5]
siteindx = np.array(range(1, Nmst+1))
sitepotential = 2*lbd*np.cos(2*np.pi*irrb*(siteindx))
diagonals = [sitepotential,t*np.ones(Nmst-1), t*np.ones(Nmst-1)]
offset = [0,-1,1]
sys_Ham = diags(diagonals,offset,dtype='complex_').toarray()
mu_L = 0.0
mu_R = np.linspace(0.01,15.0,50)
beta_left = 1/80
beta_right = 1/180
beta_probe = 1/110
def bosonic_distribution(mu, energy, beta):#bosonic distribution function
    return 1/(np.exp(beta*(energy - mu)))
def selfenergy(gamma,site,energy):#self energy matrix
    mat =     mat = np.zeros((Nmst,Nmst),dtype = 'complex_')
    mat[site,site] =  ((gamma**2/(2*to**2))*(energy - cmath.sqrt(4*to**2-energy**2)*1j))
    return mat
def specden(gamma,energy):#spectral density matrix(-2Im(sigma))
    mat =  -2*((gamma**2/(2*to**2))*(energy - cmath.sqrt(4*to**2-energy**2)*1j)).imag
    return mat
def Green_func(energy):#green's function
    mat = energy*np.identity(Nmst) - sys_Ham - selfenergy(sitegammastrn[0],sitegammaindx[0],energy) - \
    selfenergy(sitegammastrn[1],sitegammaindx[1],energy) - selfenergy(sitegammastrn[2],sitegammaindx[2],energy)
    return (np.linalg.det(mat)/t)
def advan_gren(energy):
    mat = np.transpose(np.conjugate(Green_func(energy)))
    return mat
def transmissionprob(sitstrn1, sitstrn2, energy):
    spcdn1 = specden(sitstrn1, energy)
    retgre = Green_func(energy)
    spcdn2 = specden(sitstrn2, energy)
    advgre = advan_gren(energy)
    mat = (spcdn1*spcdn2)/(abs(retgre)**2)
    return mat

In [11]:
point = np.linspace(-5,5, 1500)
first_trans_left = transmissionprob(sitegammastrn[2], sitegammastrn[0],point)
second_trans_right = transmissionprob(sitegammastrn[2], sitegammastrn[1],point)
def current_val(energy,mu_p,mu_l,mu_r):#integral equation
    first_trans = transmissionprob(sitegammastrn[2], sitegammastrn[0],energy)
    left_bath = bosonic_distribution(mu_L, energy, beta_left)
    probe = bosonic_distribution(mu_p, energy, beta_probe)
    right_bath = bosonic_distribution(mu_R, energy, beta_right)
    second_trans = transmissionprob(sitegammastrn[2], sitegammastrn[1],energy)
    return first_trans*(probe - left_bath) + second_trans*(probe - right_bath)


current_val2 = np.vectorize(current_val)
def current_int(mu_p,mu_l,mu_r):
    I = current_val2(point,mu_p,mu_l,mu_r)
    return(integrate.simps(I,point))
#for i in range(0,len(mu_p)):
#    I = current_val2(point,mu_p[i],0.0,2.5)
#    plt.plot(point,I)
#    plt.show()
#    print(integrate.simps(I,point))
ion = np.vectorize(current_int)


Derivative calculation

In [14]:
def current_val(energy,mu_p,mu_l,mu_r):#integral equation
    first_trans = transmissionprob(sitegammastrn[2], sitegammastrn[0],energy)
    left_bath = bosonic_distribution(mu_l, energy, beta_left)
    probe = bosonic_distribution(mu_p, energy, beta_probe)
    right_bath = bosonic_distribution(mu_r, energy, beta_right)
    second_trans = transmissionprob(sitegammastrn[2], sitegammastrn[1],energy)
    return first_trans*(probe - left_bath) + second_trans*(probe - right_bath)
def current_int(mu_p,mu_l,mu_r):
    point = np.linspace(-5.0,5.0,1500)
    current_val2 = np.vectorize(current_val)
    I = current_val2(point,mu_p,mu_l,mu_r)
    return(integrate.simps(I,point))
print(optimize.newton(lambda x: current_int(x,1.5,2.5),(1.5+2.5)/2))
def current_full(energy, mu_l, mu_r):
    mu_poi = optimize.newton(lambda x: current_int(x,mu_l,mu_r),(mu_l+mu_r)/2)
    first_trans = transmissionprob(sitegammastrn[1], sitegammastrn[0],energy)
    left_bath = bosonic_distribution(mu_l, energy, beta_left)
    probe = bosonic_distribution(mu_poi, energy, beta_probe)
    right_bath = bosonic_distribution(mu_r, energy, beta_right)
    second_trans = transmissionprob(sitegammastrn[1], sitegammastrn[2],energy)
    return first_trans*(right_bath - left_bath) + second_trans*(right_bath - probe)
point = np.linspace(-5.0,5.0,1500) 
current_temp = np.vectorize(current_full)
def current_full2(mu_l,mu_r):  
    point = np.linspace(-5.0,5.0,1500) 
    current_temp = np.vectorize(current_full)
    I = current_temp(point,mu_l, mu_r)
    return(integrate.simps(I,point))

for i in range(len(mu_R)):
    plot = []
    print(i)
    plot.append(current_full2(mu_L,mu_R[i]))
    print(i)
np.savetxt('current_file' + '.txt', plot )
np.savetxt('x_axis.txt', mu_R)

1.7960641041224596
0
